In [1]:
import requests
import re
from bs4 import BeautifulSoup
import math
import pandas as pd
import time
import random

https://www.scrapingdog.com/blog/scrape-linkedin-jobs/

https://www.geeksforgeeks.org/using-jupyter-notebook-in-virtual-environment/

# Target URL
Follow the steps to produce a target URL:
1. Goto Linkedin and search for a job with any filters you want.
2. Open up the Developer inspector (Use Google Chrome) and scroll down until the next set of jobs load.
3. In the network tab there should be a "search" request when it loads the new jobs.
4. Grab that URL in the search request.
5. replace the last NUMBER in the URL with a {} in order to utilize a .format() for looping later. ie 'start=0' -> 'start={}'

In [2]:
target_url = 'https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Data%2BScientist&location=United%2BStates&geoId=103644278&f_TPR=r86400&f_JT=F&f_E=2%2C4&f_WT=2&currentJobId=3692555868&position=2&pageNum=2&start={}'

# Job ID Loop
Used to get the Job ID to appened to the job URL for crawling

In [3]:
def job_id_scrape(target_url, num_of_jobs):
    l=[] #Empty list to store job IDs. These IDs are used to create the URLs for specific job searches later
    a = 0 #Used for the URL page number
    pattern = 'jobPosting:(?P<job_id>\d+)' #Regex patter to get the job ID
    for i in range(0, math.ceil(num_of_jobs/25)): #Use math.ceil() to round off the loop, the numerator can be found by looking at the filters when building the target URL.
        res = requests.get(target_url.format(a)) #Get request against the target URL and a .format() in order to provide specific page number.
        a += 25
        soup = BeautifulSoup(res.text, 'html.parser')
        all_jobs = soup.find_all('li') #The 'li' tag contains the job ID
        #print(len(all_jobs)) #Debug
        for j in range(0, len(all_jobs)): #Loop through all b4 'li' list
            job_id = re.search(pattern, str(all_jobs[j])) #Using Regex to search for the ID
            l.append(int(job_id.groups()[0])) #Appened the ID and casting it as int for later use.
    return l

In [16]:
job_ids_list = job_id_scrape(target_url, 1000)

In [18]:
len(job_ids_list)

975

# Job Loop
This is made outside of a function on purpose because if the function fails at any step it won't return a DF.

In [62]:
df = {'job_title' : [], 'company' : [], 'job_info' : [], 'when_posted' : [], 'num_applicants' : [],
      'seniority' : [], 'employment_type' : [], 'job_function' : [], 'industry' : [], 'job_id' : []}
target_job_url = 'https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{}/' #format of the URL. The {} is where the jobID gets formated to.
for i in range(0, len(job_ids_list)): # Main loop
    res_job = requests.get(target_job_url.format(job_ids_list[i]))
    soup = BeautifulSoup(res_job.text, 'html.parser')
    try:
        df['job_title'].append(soup.find('div',{'class':'top-card-layout__entity-info'}).find('a').text.strip()) #Title
    except:
        df['job_title'].append(None)
    try:
        df['company'].append(soup.find('div',{'class':'top-card-layout__card'}).find('a').find('img').get('alt')) #Company
    except:
        df['company'].append(None)
    try:
        df['job_info'].append(soup.find('div',{'class':'show-more-less-html__markup'}).text.strip()) #Bulk, might need to find a way to clean this up.
    except:
        df['job_info'].append(None)
    try:
        df['when_posted'].append(soup.find('span',{'class':'posted-time-ago__text'}).text.replace('\n', '').strip()) #When was it posted
    except:
        df['when_posted'].append(None)
    num_of_apps1 = soup.find('figcaption',{'class':'num-applicants__caption'})  #number of applicants scape attempt1
    if num_of_apps1 == None: #Check first soup, if it fails check second soup
        num_of_apps2 = soup.find('span', class_='num-applicants__caption topcard__flavor--metadata topcard__flavor--bullet') #number of applicants scrape attempt2
        if num_of_apps2 == None: #Check second soup, if it fails, append None
            df['num_applicants'].append(None) #if nosoup works just append None
        else:
            df['num_applicants'].append(num_of_apps2.text.replace('\n', '').strip()) #If first soup failed and second soup passed, append second soup
    else:
        df['num_applicants'].append(num_of_apps1.text.replace('\n', '').strip()) #If first soup works, append first soup
    job_attributes = soup.find('ul',{'class':'description__job-criteria-list'}) #Job attributes
    if job_attributes == None: #Check to see if there is soup. If not, pass.
        df['seniority'].append(None)
        df['employment_type'].append(None)
        df['job_function'].append(None)
        df['industry'].append(None)
    else:
        job_attributes = job_attributes.text.replace('\n', '').strip()
        if 'Seniority level' in job_attributes:
            pattern = 'Seniority level\s+(?P<seniority>[a-zA-z!@#$%^&*()-]+\s[a-zA-z!@#$%^&*()-]*)'
            df['seniority'].append(re.search(pattern, test_string).groups()[0])
        else:
            df['seniority'].append(None)
        if 'Employment type' in job_attributes:
            pattern = 'Employment type\s+(?P<employment_type>[a-zA-z!@#$%^&*()-]+\s[a-zA-z!@#$%^&*()-]*)'
            df['employment_type'].append(re.search(pattern, test_string).groups()[0])
        else:
            df['employment_type'].append(None)
        if 'Job function' in job_attributes:
            pattern = 'Job function\s+(?P<job_function>[a-zA-z!@#$%^&*()-]+\s[a-zA-z!@#$%^&*()-]*)'
            df['job_function'].append(re.search(pattern, test_string).groups()[0])
        else:
            df['job_function'].append(None)
        if 'Industries' in job_attributes:
            pattern = 'Industries\s+(?P<industries>[a-zA-z!@#$%^&*()-]+\s[a-zA-z!@#$%^&*()-]*)'
            df['industry'].append(re.search(pattern, test_string).groups()[0])
        else:
            df['industry'].append(None)
    print(f'Obatined info for id: {job_ids_list[i]}')
    df['job_id'].append(job_ids_list[0])
    job_ids_list.pop(0)
    time.sleep(random.choice(range(3,5,1))) # Sleep to avoid IP block

Obatined info for id: 3691869782
Obatined info for id: 3696159081
Obatined info for id: 3692553497
Obatined info for id: 3695130693
Obatined info for id: 3696143316
Obatined info for id: 3691836860


KeyboardInterrupt: 

In [63]:
df = pd.DataFrame(df)
df.head()

,job_title,company,job_info,when_posted,num_applicants,seniority,employment_type,job_function,industry,job_id
0,"Data Scientist, gt.school (Remote) - $60,000/y...",Crossover,Crossover is the world's #1 source of full-tim...,19 hours ago,Be among the first 25 applicants,Entry level,Full-time,Engineering and,Software Development,3691869782
1,Research Scientist,Deepgram,Company OverviewDeepgram is a foundational AI ...,1 hour ago,Be among the first 25 applicants,Entry level,Full-time,Engineering and,Software Development,3696142461
2,Remote - Application Developer,Sonitalent Corp,Job Role: Application DeveloperDuration: 6 mon...,2 hours ago,Be among the first 25 applicants,Entry level,Full-time,Engineering and,Software Development,3696159081
3,Associate Software Engineer (Remote),"Patterson Companies, Inc.",Dolphin Imaging Systems has a great opportunit...,19 hours ago,169 applicants,Entry level,Full-time,Engineering and,Software Development,3689951639
4,Junior Mobile App Developer,Pattern Learning AI - Career & Tech Recruitmen...,This is a remote position.Junior Mobile App De...,2 hours ago,Be among the first 25 applicants,Entry level,Full-time,Engineering and,Software Development,3692553497


In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   job_title        6 non-null      object
 1   company          6 non-null      object
 2   job_info         6 non-null      object
 3   when_posted      6 non-null      object
 4   num_applicants   6 non-null      object
 5   seniority        6 non-null      object
 6   employment_type  6 non-null      object
 7   job_function     6 non-null      object
 8   industry         6 non-null      object
 9   job_id           6 non-null      int64 
dtypes: int64(1), object(9)
memory usage: 612.0+ bytes
